In [1]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/\(데이콘\)운동동작분류/data ./

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Conv1D, MaxPool1D, SeparableConv1D, AvgPool1D, MaxPool2D, GlobalAvgPool2D
from tensorflow.keras.layers import Flatten, Activation, Dropout, BatchNormalization, Input, Add, GlobalAvgPool1D, Subtract, Reshape, Concatenate, Permute
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [10]:
df = pd.read_csv('./data/train_features.csv')
train_x = df.values[:,2:].reshape(-1,600,6)
print(train_x.shape)
# df

(3125, 600, 6)


In [11]:
df = pd.read_csv('./data/train_labels.csv')
train_y = df.values[:,1].astype(np.int)
print(train_y.shape)
# df

(3125,)


In [ ]:
mean_train_x = []
for i in range(61):
  inds = np.where(train_y==i)[0]
  data = np.mean(train_x[inds], axis=0).tolist()
  mean_train_x.append(data)

mean_train_x = np.array(mean_train_x)
mean_train_x.shape

(61, 600, 6)

In [ ]:
# long_mean_train_x = []
# for label in train_y:
#   one_mean = mean_train_x[label].tolist()
#   long_mean_train_x.append(one_mean)
# long_mean_train_x = np.array(long_mean_train_x)
# long_mean_train_x.shape

(3125, 600, 6)

In [ ]:
bool_train_y = []
for i in range(61):
  equ = train_y == 1
  bool_train_y.append(equ.tolist())
bool_train_y = np.array(bool_train_y, dtype=np.int8)
bool_train_y.shape

(61, 3125)

In [13]:
def conv1d_bn(x, filters, kernel_size, padding='same', strides=1, activation='elu', weight_decay=1e-5, bn = True):
    x = Conv1D(filters, kernel_size, padding=padding, strides=strides, kernel_regularizer=l2(weight_decay))(x)
    if bn: x = BatchNormalization()(x)
    if activation: x = Activation(activation)(x)
    return x

def sepconv1d_bn(x, filters, kernel_size, padding='same', strides=1, activation='elu', weight_decay=1e-5, depth_multiplier=1, bn = True):
    x = SeparableConv1D(filters, kernel_size, padding=padding, strides=strides, depth_multiplier=depth_multiplier, depthwise_regularizer=l2(weight_decay), pointwise_regularizer=l2(weight_decay))(x)
    if bn: x = BatchNormalization()(x)
    if activation: x = Activation(activation)(x)
    return x

In [ ]:
fm_inp = Input(shape=(600,6))
x = fm_inp

res = conv1d_bn(x, 64, 1, strides=2, activation=None)
x = conv1d_bn(x, 64, 3, activation='elu')
x = conv1d_bn(x, 64, 3, activation=None)
x = MaxPool1D(pool_size=2, strides=2, padding='same')(x)
x = Add()([x, res])
x = Activation('elu')(x)

res = conv1d_bn(x, 128, 1, strides=2, activation=None)
x = conv1d_bn(x, 128, 3, activation='elu')
x = conv1d_bn(x, 128, 3, activation=None)
x = MaxPool1D(pool_size=2, strides=2, padding='same')(x)
x = Add()([x, res])
x = Activation('elu')(x)

fm_out = x

feature_model = Model(fm_inp, fm_out)
# feature_model.summary()

In [ ]:
em_inp = Input(shape=(600,6))

fm_net1 = Concatenate(axis=1)([Reshape((1,150,128))(feature_model(mean_train_x[None, i])) for i in range(61)])

# fm_net1 = feature_model(mean_train_x[None, 0]) # (61,150,128)
fm_net2 = feature_model(em_inp) # (150,128)
# fm_net1 = Reshape((1, 61, 150, 128))(fm_net1)
# fm_net2 = Reshape((-1, 1, 150, 128))(fm_net2)

x = Subtract()([fm_net1, fm_net2]) # (61,150,128)

res = conv1d_bn(x, 256, 1, strides=2, activation=None)
x = conv1d_bn(x, 256, 3, activation='elu')
x = conv1d_bn(x, 256, 3, activation='elu')
x = conv1d_bn(x, 256, 3, activation=None)
# x = MaxPool1D(pool_size=2, strides=2, padding='same')(x)
x = MaxPool2D(pool_size=(1,2), strides=(1,2), padding='same')(x)
x = Add()([x, res])
x = Activation('elu')(x)

res = conv1d_bn(x, 512, 1, strides=2, activation=None)
x = conv1d_bn(x, 512, 3, activation='elu')
x = conv1d_bn(x, 512, 3, activation='elu')
x = conv1d_bn(x, 512, 3, activation=None)
# x = MaxPool1D(pool_size=2, strides=2, padding='same')(x)
x = MaxPool2D(pool_size=(1,2), strides=(1,2), padding='same')(x)
x = Add()([x, res])
x = Activation('elu')(x)

x = conv1d_bn(x, 64, 1, activation='elu')
x = Reshape((61,-1))(x)

x = conv1d_bn(x, 64, 1, activation='elu')
x = conv1d_bn(x, 1, 1, activation='elu')
x = Flatten()(x)
# x = Reshape((61))(x)
# x = Permute((2,1))(x)

x = Dense(61, activation='softmax')(x)

em_out = x

isequal_model = Model(inputs=em_inp, outputs=em_out, )
isequal_model.summary()

In [ ]:
def lr_scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * 0.95

callback1 = LearningRateScheduler(lr_scheduler)

In [ ]:
optimizer = SGD(lr=0.001, momentum=0.9)
# model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['acc'])
isequal_model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])
# model.compile(optimizer, loss=[focal_loss], metrics=['acc'])

In [ ]:
part_x, valid_x, part_y, valid_y = train_test_split(train_x, train_y, test_size=0.1, random_state=777)

In [ ]:
history = isequal_model.fit(
    part_x, to_categorical(part_y, num_classes=61), 
    # train_x, to_categorical(train_y, num_classes=61), 
    validation_data=(valid_x, to_categorical(valid_y, num_classes=61)), 
    validation_split=0, epochs=70, batch_size=12, callbacks=[callback1])

Epoch 1/70


KeyboardInterrupt: ignored